# Automated Car in Project Cars

Work based on the project madeby Sentdex in his blog: https://pythonprogramming.net/game-frames-open-cv-python-plays-gta-v/
    

## Main formulas

In [1]:
import numpy as np
from numpy import ones,vstack
from numpy.linalg import lstsq
from PIL import ImageGrab
import cv2
import time
from IPython.display import Image
import ctypes
from statistics import mean
import win32gui, win32ui, win32con, win32api
import os
import pandas as pd
from collections import Counter
from random import shuffle
from PIL import Image as im
from keras_preprocessing.image import ImageDataGenerator

In [2]:
def screen_record(): 
    last_time = time.time()
    while(True):
        # 800x600 windowed mode
        printscreen =  np.array(ImageGrab.grab(bbox=(0,40,800,640)))
        print('loop took {} seconds'.format(time.time()-last_time))
        last_time = time.time()
        cv2.imshow('window',cv2.cvtColor(printscreen, cv2.COLOR_BGR2RGB))
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

In [3]:
def process_img(image):
    original_image = image
    # convert to gray
    processed_img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # edge detection
    processed_img =  cv2.Canny(processed_img, threshold1 = 200, threshold2=300)
    
    processed_img = cv2.GaussianBlur(processed_img,(5,5),0)
    
    vertices = np.array([[10,500],[10,300],[300,200],[500,200],[800,300],[800,500],
                         ], np.int32)

    processed_img = roi(processed_img, [vertices])

    # more info: http://docs.opencv.org/3.0-beta/doc/py_tutorials/py_imgproc/py_houghlines/py_houghlines.html
    #                                     rho   theta   thresh  min length, max gap:        
    lines = cv2.HoughLinesP(processed_img, 1, np.pi/180, 180,      20,       15)
    m1 = 0
    m2 = 0
    try:
        l1, l2, m1,m2 = draw_lanes(original_image,lines)
        cv2.line(original_image, (l1[0], l1[1]), (l1[2], l1[3]), [0,255,0], 30)
        cv2.line(original_image, (l2[0], l2[1]), (l2[2], l2[3]), [0,255,0], 30)
    except Exception as e:
        print(str(e))
        pass
    try:
        for coords in lines:
            coords = coords[0]
            try:
                cv2.line(processed_img, (coords[0], coords[1]), (coords[2], coords[3]), [255,0,0], 3)
                
                
            except Exception as e:
                print(str(e))
    except Exception as e:
        pass

    return processed_img,original_image, m1, m2

In [4]:
def draw_lines(img, lines):
    try:
        for line in lines:
            coords = line[0]
            cv2.line(img, (coords[0],coords[1]), (coords[2],coords[3]), [255,255,255], 3)
    except:
        pass

In [5]:
def roi(img, vertices):
    
    #blank mask:
    mask = np.zeros_like(img)   
    
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, 255)
    
    #returning the image only where mask pixels are nonzero
    masked = cv2.bitwise_and(img, mask)
    return masked

In [6]:
def draw_lanes(img, lines, color=[0, 255, 255], thickness=3):

    # if this fails, go with some default line
    try:

        # finds the maximum y value for a lane marker 
        # (since we cannot assume the horizon will always be at the same point.)

        ys = []  
        for i in lines:
            for ii in i:
                ys += [ii[1],ii[3]]
        min_y = min(ys)
        max_y = 600
        new_lines = []
        line_dict = {}

        for idx,i in enumerate(lines):
            for xyxy in i:
                # These four lines:
                # modified from http://stackoverflow.com/questions/21565994/method-to-return-the-equation-of-a-straight-line-given-two-points
                # Used to calculate the definition of a line, given two sets of coords.
                x_coords = (xyxy[0],xyxy[2])
                y_coords = (xyxy[1],xyxy[3])
                A = vstack([x_coords,ones(len(x_coords))]).T
                m, b = lstsq(A, y_coords)[0]

                # Calculating our new, and improved, xs
                x1 = (min_y-b) / m
                x2 = (max_y-b) / m

                line_dict[idx] = [m,b,[int(x1), min_y, int(x2), max_y]]
                new_lines.append([int(x1), min_y, int(x2), max_y])

        final_lanes = {}

        for idx in line_dict:
            final_lanes_copy = final_lanes.copy()
            m = line_dict[idx][0]
            b = line_dict[idx][1]
            line = line_dict[idx][2]
            
            if len(final_lanes) == 0:
                final_lanes[m] = [ [m,b,line] ]
                
            else:
                found_copy = False

                for other_ms in final_lanes_copy:

                    if not found_copy:
                        if abs(other_ms*1.2) > abs(m) > abs(other_ms*0.8):
                            if abs(final_lanes_copy[other_ms][0][1]*1.2) > abs(b) > abs(final_lanes_copy[other_ms][0][1]*0.8):
                                final_lanes[other_ms].append([m,b,line])
                                found_copy = True
                                break
                        else:
                            final_lanes[m] = [ [m,b,line] ]

        line_counter = {}

        for lanes in final_lanes:
            line_counter[lanes] = len(final_lanes[lanes])

        top_lanes = sorted(line_counter.items(), key=lambda item: item[1])[::-1][:2]

        lane1_id = top_lanes[0][0]
        lane2_id = top_lanes[1][0]

        def average_lane(lane_data):
            x1s = []
            y1s = []
            x2s = []
            y2s = []
            for data in lane_data:
                x1s.append(data[2][0])
                y1s.append(data[2][1])
                x2s.append(data[2][2])
                y2s.append(data[2][3])
            return int(mean(x1s)), int(mean(y1s)), int(mean(x2s)), int(mean(y2s)) 

        l1_x1, l1_y1, l1_x2, l1_y2 = average_lane(final_lanes[lane1_id])
        l2_x1, l2_y1, l2_x2, l2_y2 = average_lane(final_lanes[lane2_id])

        return [l1_x1, l1_y1, l1_x2, l1_y2], [l2_x1, l2_y1, l2_x2, l2_y2], lane1_id, lane2_id
    except Exception as e:
        print(str(e))

In [7]:
# direct inputs
# source to this solution and code:
# http://stackoverflow.com/questions/14489013/simulate-python-keypresses-for-controlling-a-game
# http://www.gamespp.com/directx/directInputKeyboardScanCodes.html

SendInput = ctypes.windll.user32.SendInput


W = 0x11
A = 0x1E
S = 0x1F
D = 0x20

# C struct redefinitions 
PUL = ctypes.POINTER(ctypes.c_ulong)
class KeyBdInput(ctypes.Structure):
    _fields_ = [("wVk", ctypes.c_ushort),
                ("wScan", ctypes.c_ushort),
                ("dwFlags", ctypes.c_ulong),
                ("time", ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

class HardwareInput(ctypes.Structure):
    _fields_ = [("uMsg", ctypes.c_ulong),
                ("wParamL", ctypes.c_short),
                ("wParamH", ctypes.c_ushort)]

class MouseInput(ctypes.Structure):
    _fields_ = [("dx", ctypes.c_long),
                ("dy", ctypes.c_long),
                ("mouseData", ctypes.c_ulong),
                ("dwFlags", ctypes.c_ulong),
                ("time",ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

class Input_I(ctypes.Union):
    _fields_ = [("ki", KeyBdInput),
                 ("mi", MouseInput),
                 ("hi", HardwareInput)]

class Input(ctypes.Structure):
    _fields_ = [("type", ctypes.c_ulong),
                ("ii", Input_I)]

# Actuals Functions

def PressKey(hexKeyCode):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.ki = KeyBdInput( 0, hexKeyCode, 0x0008, 0, ctypes.pointer(extra) )
    x = Input( ctypes.c_ulong(1), ii_ )
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

def ReleaseKey(hexKeyCode):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.ki = KeyBdInput( 0, hexKeyCode, 0x0008 | 0x0002, 0, ctypes.pointer(extra) )
    x = Input( ctypes.c_ulong(1), ii_ )
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))

In [8]:
def straight():
    PressKey(W)
    ReleaseKey(A)
    ReleaseKey(D)

def left():
    PressKey(A)
    ReleaseKey(W)
    ReleaseKey(D)
    ReleaseKey(A)

def right():
    PressKey(D)
    ReleaseKey(A)
    ReleaseKey(W)
    ReleaseKey(D)

def slow_ya_roll():
    ReleaseKey(W)
    ReleaseKey(A)
    ReleaseKey(D)

In [9]:
def grab_screen(region=None):

    hwin = win32gui.GetDesktopWindow()

    if region:
            left,top,x2,y2 = region
            width = x2 - left + 1
            height = y2 - top + 1
    else:
        width = win32api.GetSystemMetrics(win32con.SM_CXVIRTUALSCREEN)
        height = win32api.GetSystemMetrics(win32con.SM_CYVIRTUALSCREEN)
        left = win32api.GetSystemMetrics(win32con.SM_XVIRTUALSCREEN)
        top = win32api.GetSystemMetrics(win32con.SM_YVIRTUALSCREEN)

    hwindc = win32gui.GetWindowDC(hwin)
    srcdc = win32ui.CreateDCFromHandle(hwindc)
    memdc = srcdc.CreateCompatibleDC()
    bmp = win32ui.CreateBitmap()
    bmp.CreateCompatibleBitmap(srcdc, width, height)
    memdc.SelectObject(bmp)
    memdc.BitBlt((0, 0), (width, height), srcdc, (left, top), win32con.SRCCOPY)
    
    signedIntsArray = bmp.GetBitmapBits(True)
    img = np.fromstring(signedIntsArray, dtype='uint8')
    img.shape = (height,width,4)

    srcdc.DeleteDC()
    memdc.DeleteDC()
    win32gui.ReleaseDC(hwin, hwindc)
    win32gui.DeleteObject(bmp.GetHandle())

    return cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)

In [10]:
import win32api as wapi
import time

keyList = ["\b"]
for char in "ABCDEFGHIJKLMNOPQRSTUVWXYZ 123456789,.'APS$/\\":
    keyList.append(char)

def key_check():
    keys = []
    for key in keyList:
        if wapi.GetAsyncKeyState(ord(key)):
            keys.append(key)
    return keys

In [11]:
def keys_to_output(keys):
    '''
    Convert keys to a ...multi-hot... array

    [A,W,D] boolean values.
    '''
    output = [0,0,0]
    
    if 'A' in keys:
        output[0] = 1
    elif 'D' in keys:
        output[2] = 1
    else:
        output[1] = 1
    return output

In [12]:
file_name = 'training_data.npy'

if os.path.isfile(file_name):
    print('File exists, loading previous data!')
    training_data = list(np.load(file_name, allow_pickle=True))
else:
    print('File does not exist, starting fresh!')
    training_data = []

File exists, loading previous data!


In [13]:
def main():

    for i in list(range(4))[::-1]:
        print(i+1)
        time.sleep(1)
        
    while(True):
        # 800x600 windowed mode
        screen = grab_screen(region=(0,40,800,640))
        last_time = time.time()
        screen = cv2.cvtColor(screen, cv2.COLOR_BGR2GRAY)
        # resize to something a bit more acceptable for a CNN
        screen = cv2.resize(screen, (80,60))
        keys = key_check()
        output = keys_to_output(keys)
        training_data.append([screen,output])
        
        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

        if len(training_data) % 500 == 0:
            print(len(training_data))
            np.save(file_name,training_data)

In [14]:
# main()

---------------

## Data Management

In [12]:
train_data = np.load(r'../../data/training_data.npy', allow_pickle=True)

df = pd.DataFrame(train_data)
print(df.head())
print(Counter(df[1].apply(str)))

                                                   0          1
0  [[237, 236, 235, 234, 233, 232, 230, 228, 226,...  [0, 1, 0]
1  [[237, 236, 235, 234, 233, 232, 230, 228, 227,...  [0, 1, 0]
2  [[237, 236, 235, 234, 233, 232, 230, 228, 227,...  [0, 1, 0]
3  [[237, 236, 235, 234, 233, 232, 230, 228, 227,...  [0, 1, 0]
4  [[237, 236, 235, 234, 232, 232, 230, 228, 227,...  [0, 1, 0]
Counter({'[0, 1, 0]': 97046, '[0, 0, 1]': 29425, '[1, 0, 0]': 19529})


In [15]:
lefts = []
rights = []
forwards = []

shuffle(train_data)

for data in train_data:
    img = data[0]
    choice = data[1]

    if choice == [1,0,0]:
        lefts.append([img,choice])
    elif choice == [0,1,0]:
        forwards.append([img,choice])
    elif choice == [0,0,1]:
        rights.append([img,choice])
    else:
        print('no matches')

forwards = forwards[:len(lefts)]
lefts = lefts[:len(forwards)]
rights = rights[:len(lefts)]



final_data = forwards + lefts + rights
shuffle(final_data)

np.save(r'../../data/training_data_v4.npy', final_data)

C:\Users\Alvaro\miniconda3\envs\prostate\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [16]:
train_data_v2 = np.load(r'../../data/training_data_v4.npy', allow_pickle=True)

df_2 = pd.DataFrame(train_data_v2)
print(df_2.head())
print(Counter(df_2[1].apply(str)))

                                                   0          1
0  [[11, 25, 22, 28, 29, 28, 25, 30, 30, 29, 22, ...  [0, 1, 0]
1  [[96, 74, 101, 164, 171, 171, 170, 169, 169, 1...  [1, 0, 0]
2  [[187, 186, 185, 185, 183, 183, 182, 181, 181,...  [0, 0, 1]
3  [[251, 251, 250, 250, 249, 249, 248, 248, 247,...  [1, 0, 0]
4  [[60, 113, 74, 48, 62, 52, 54, 42, 52, 101, 16...  [1, 0, 0]
Counter({'[0, 1, 0]': 16917, '[1, 0, 0]': 16917, '[0, 0, 1]': 16917})


In [51]:
def array_to_img(array, name):
    data = im.fromarray(array)
    save_dir = "images/"
    data.save(save_dir + f'{name}.png')

In [52]:
#No need to do it everytime, just if the images are not created from the numpy
# df_3 = df_2[0]

# for i in range(len(df_3)):
#     name = i
#     array_to_img(df_3[i], name)

In [53]:
cleaning_df = pd.DataFrame(df_2[[1]])
cleaning_df.head()

,1
0,"[1, 0, 0]"
1,"[0, 0, 1]"
2,"[0, 0, 1]"
3,"[0, 0, 1]"
4,"[0, 0, 1]"


In [54]:
cleaning_df['name'] = np.arange(cleaning_df.shape[0])

cleaning_df['name'] = cleaning_df['name'].apply(str)

cleaning_df['result'] = cleaning_df[1]

cleaning_df.head()

,1,name,result
0,"[1, 0, 0]",0,"[1, 0, 0]"
1,"[0, 0, 1]",1,"[0, 0, 1]"
2,"[0, 0, 1]",2,"[0, 0, 1]"
3,"[0, 0, 1]",3,"[0, 0, 1]"
4,"[0, 0, 1]",4,"[0, 0, 1]"


In [55]:
def find_1(value):
    result = 0
    if value == [0,0,1]:
        result = '2'
    elif value == [0,1,0]:
        result = '1'
    else:
        result = '0'
    return result


cleaning_df['result'] = cleaning_df[1].apply(lambda x: find_1(x)).astype(int)
cleaning_df.head()

,1,name,result
0,"[1, 0, 0]",0,0
1,"[0, 0, 1]",1,2
2,"[0, 0, 1]",2,2
3,"[0, 0, 1]",3,2
4,"[0, 0, 1]",4,2


In [56]:
cleaning_df['path'] = cleaning_df['name'] + '.png'
cleaning_df.head()

,1,name,result,path
0,"[1, 0, 0]",0,0,0.png
1,"[0, 0, 1]",1,2,1.png
2,"[0, 0, 1]",2,2,2.png
3,"[0, 0, 1]",3,2,3.png
4,"[0, 0, 1]",4,2,4.png


In [57]:
train_df = pd.DataFrame(cleaning_df[['path', 'result']])

train_df.head()

,path,result
0,0.png,0
1,1.png,2
2,2.png,2
3,3.png,2
4,4.png,2


In [58]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25377 entries, 0 to 25376
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    25377 non-null  object
 1   result  25377 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 297.5+ KB


------

## Image Treatment

In [59]:
datagen = ImageDataGenerator(rescale=1./255.,
                             validation_split=0.20,
                             rotation_range=10, # rotation
                             width_shift_range=0.2, # horizontal shift
                             height_shift_range=0.2, # vertical shift
                             zoom_range=0.2, # zoom
                             horizontal_flip=True, # horizontal flip
                             brightness_range=[0.2,1.2]) # brightness)

In [60]:
save_dir = "images/"

train_generator  = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=save_dir,
    x_col="path",
    y_col="result",
    target_size=(80, 60),
    color_mode="grayscale",
    batch_size=64,
    save_to_dir='png_images_resized/',
    save_format="png",
    class_mode="raw",
    subset='training',
    shuffle = True
)


validation_generator  = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=save_dir,
    x_col="path",
    y_col="result",
    target_size=(80, 60),
    color_mode="grayscale",
    batch_size=64, 
    save_to_dir='png_images_resized/',
    save_format="png",
    class_mode="raw",
    subset='validation',
    shuffle = True
)

Found 20302 validated image filenames.
Found 5075 validated image filenames.


---------

## Modeling

In [61]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.4.0
Eager mode:  True
GPU is available


In [62]:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=4)

In [63]:
# model = tf.keras.Sequential()


# model.add(tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(80, 60, 1)))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# model.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# model.add(tf.keras.layers.Dropout(0.2))

# model.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

# model.add(tf.keras.layers.Dropout(0.2))

# model.add(tf.keras.layers.Flatten())

# model.add(tf.keras.layers.Dense(64, activation="relu"))

# model.add(tf.keras.layers.Dropout(0.2))

# model.add(tf.keras.layers.Dense(32, activation="relu"))

# model.add(tf.keras.layers.Dropout(0.2))

# model.add(tf.keras.layers.Dense(16, activation="relu"))

# model.add(tf.keras.layers.Dropout(0.2))

# model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

In [68]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(16,(3,3),input_shape=(80,60,1)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size =(2,2)))
model.add(tf.keras.layers.Conv2D(32,(3,3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size =(2,2)))
model.add(tf.keras.layers.Conv2D(64,(3,3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size =(2,2)))
model.add(tf.keras.layers.Conv2D(128,(3,3)))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size =(2,2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(32))
model.add(tf.keras.layers.Activation('relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Activation('softmax'))


In [75]:
# opt = keras.optimizers.Adam(learning_rate=1e-3)
# model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.compile(optimizer='adam', 
              loss="categorical_crossentropy", 
              metrics=['accuracy'])

In [76]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 78, 58, 16)        160       
_________________________________________________________________
activation_16 (Activation)   (None, 78, 58, 16)        0         
_________________________________________________________________
batch_normalization_8 (Batch (None, 78, 58, 16)        64        
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 39, 29, 16)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 37, 27, 32)        4640      
_________________________________________________________________
activation_17 (Activation)   (None, 37, 27, 32)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 37, 27, 32)       

--------

## Model Training

In [77]:
history = model.fit(
    train_generator,
    validation_data = validation_generator,
    batch_size=32,
    epochs = 8,
    callbacks=[callback])

Epoch 1/8
318/318 [==============================] - 62s 193ms/step - loss: 0.0000e+00 - accuracy: 0.3351 - val_loss: 0.0000e+00 - val_accuracy: 0.3304
Epoch 2/8
318/318 [==============================] - 61s 192ms/step - loss: 0.0000e+00 - accuracy: 0.3363 - val_loss: 0.0000e+00 - val_accuracy: 0.3304
Epoch 3/8
318/318 [==============================] - 61s 193ms/step - loss: 0.0000e+00 - accuracy: 0.3410 - val_loss: 0.0000e+00 - val_accuracy: 0.3304
Epoch 4/8
318/318 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.3350

KeyboardInterrupt: 